In [ ]:
# Важно! Не копируйте сразу целиком. Попробуйте что-нибудь изменить, а потом копировать себе

In [1]:
import numpy as np
import pandas as pd

import os

from PIL import Image

import torch
from torch.utils.data import DataLoader, Dataset

import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

# Load data module

In [2]:
class PawpularDataset(Dataset):
    def __init__(self, images_path, features_path, transform, test=False):
        self.images_path = images_path
        self.df = pd.read_csv(features_path)
        if test:
            self.df['Pawpularity'] = np.nan # можно попробовать написать удачнее
        
        self.imgs = self.df['Id']
        self.features = self.df.drop(['Id', 'Pawpularity'], axis=1)
        self.targets = self.df['Pawpularity']
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.images_path, img_id+'.jpg')).convert('RGB')
        img = self.transform(img)
        
        features = torch.FloatTensor(self.features.values[index])
        targets = self.targets.values[index]
        
        return img, features, targets # сделать сразу torch.tensor(targets, dtype=torch.float) и т.д.

# Model

In [3]:
class PawpularCNN(nn.Module):
    def __init__(self, input_size, dense_size, hidden_size, num_classes):
        super(PawpularCNN, self).__init__()
        self.resnet = models.resnet18(pretrained=False)
        self.resnet.load_state_dict(
            torch.load("../input/pretrained-pytorch-models/resnet18-5c106cde.pth")
        ) # перед этим справа нажать на кнопку "+ Add data" и через поиск добавить предобученные веса
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, hidden_size)
        self.dense = nn.Linear(input_size, dense_size) # либо менять размерность, либо убрать один слой 
        
        self.fc = nn.Linear(hidden_size+dense_size, num_classes)
        
        self.relu = nn.ReLU()
        
    def forward(self, images, features):
        for param in self.resnet.parameters():
            param.requires_grad = False
                
        resnet_features = self.relu(self.resnet(images))
        dense_features = self.relu(self.dense(features))
        
        all_features = torch.cat((resnet_features, dense_features), dim=1) # признаки из ResNet + бинарные
        all_features = torch.flatten(all_features, 1)
        
        return self.fc(all_features)

# Parameters

In [4]:
transform = transforms.Compose(
  [
   transforms.Resize((256,256)),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
  ]
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 12
hidden_size = 100
dense_size = 100
num_classes = 1
learning_rate = 3e-4
num_epochs = 10

In [5]:
torch.manual_seed(42)

trainset = PawpularDataset(images_path='../input/petfinder-pawpularity-score/train', 
                           features_path='../input/petfinder-pawpularity-score/train.csv', 
                           transform=transform)

num_examples = len(trainset)
val_len = round(0.33*num_examples)
train_len = num_examples - val_len

train, validation = torch.utils.data.random_split(trainset, [train_len, val_len])
train_loader = torch.utils.data.DataLoader(train, batch_size=100, 
                                           shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(validation, batch_size=100, 
                                         shuffle=True, num_workers=2)

In [6]:
torch.manual_seed(101)

losses = list()
model = PawpularCNN(input_size, dense_size, hidden_size, num_classes).to(device)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, num_epochs+1):
    
    for step, (img, features, target) in enumerate(train_loader):

        model.zero_grad()

        img = img.to(device)
        features = features.to(device)
        target = target.type(torch.FloatTensor) # потом убрать эту строку
        target = target.to(device)

        outputs = model(img, features)

        loss = criterion(outputs, target.unsqueeze(1))

        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        stats = 'Epoch [%d/%d], Step [%d], Loss: %.4f' % (epoch, num_epochs, step, loss.item())
        print('\r' + stats, end='')
        
    with torch.no_grad():
        val_losses = 0
        for img, features, target in val_loader:
            img = img.to(device)
            features = features.to(device)
            target = target.to(device)
            
            outputs = model(img, features)
            val_loss = criterion(outputs, target.unsqueeze(1))
            val_losses += (1/len(val_loader))*val_loss.item()
            
        print('\n Epoch [%d/%d], Val Loss: %.4f' % (epoch, num_epochs, val_losses))

In [7]:
plt.plot(losses)

In [ ]:
# Для загрузки решения:
# trainset = PawpularDataset(images_path='../input/petfinder-pawpularity-score/train', 
#                            features_path='../input/petfinder-pawpularity-score/train.csv', 
#                            transform=transform, 
#                            test=False)

# train_loader = torch.utils.data.DataLoader(trainset, batch_size=100, 
#                                            shuffle=True, num_workers=2)

# testset = PawpularDataset(images_path='../input/petfinder-pawpularity-score/test', 
#                           features_path='../input/petfinder-pawpularity-score/test.csv', 
#                           transform=transform, 
#                           test=True)

# test_loader = torch.utils.data.DataLoader(testset, batch_size=100, 
#                                           shuffle=False, num_workers=2)

# Там, где with torch.no_grad():
# убрать target = ..., val_loss = ... и val_losses ...
# поскольку предсказания будут по мини-батчам, необходимо создать для них отдельную переменную и
# на каждой итерации добавлять туда новые предсказания